In [14]:
from __future__ import print_function
import os
# Type Path
data_path = [r'C:\Users\wpghk\ai_data']

import pandas as pd
import numpy as np
from time import time
start = time()

filepath = os.sep.join(data_path + ['Orange_Telecom_Churn_Data.csv']) 
data = pd.read_csv(filepath) 
# Type your source code
data.drop(['state', 'area_code', 'phone_number'], axis=1, inplace=True)


from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier 

lb = LabelBinarizer()

for col in ['intl_plan', 'voice_mail_plan', 'churned']:
    data[col] = lb.fit_transform(data[col])
    

feature_cols = [x for x in data.columns if x != 'churned']

# Split the data into two parts with 1500 points in the test data
# This creates a generator
strat_shuff_split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)

# Get the index values from the generator
train_idx, test_idx = next(strat_shuff_split.split(data[feature_cols], data['churned']))

# Create the data sets
X_train = data.loc[train_idx, feature_cols]
y_train = data.loc[train_idx, 'churned']

X_test = data.loc[test_idx, feature_cols]
y_test = data.loc[test_idx, 'churned']

def print_score(classifier, X_train, y_train, X_test, y_test, train = True):
    print("-"*100)
    if train == True:
        y_pred = classifier.predict(X_train)
        print("Training result:\n")
        print("Accuracy Score: {0:.4f}\n".format(accuracy_score(y_train, y_pred)))
        print("Classification Report:\n{}\n".format(classification_report(y_train, y_pred)))
        print("Confusion Matrix:\n{}\n".format(confusion_matrix(y_train, y_pred)))
        res = cross_val_score(classifier, X_train, y_train, cv = 10, n_jobs= -1, scoring ="accuracy")
        print("Average Accuracy:\t{0:.4f}\n".format(res.mean()))
        print("Standard Deviation:\t{0:.4f}".format(res.std()))
    elif train == False:
        y_pred = classifier.predict(X_test)
        print("Test result:\n")
        print("Accuracy Score: {0:.4f}\n".format(accuracy_score(y_test, y_pred)))
        print("Classification Report:\n{}\n".format(classification_report(y_test, y_pred)))
        print("Confusion Matrix:\n{}\n".format(confusion_matrix(y_test, y_pred)))
        
random_forest = RandomForestClassifier(random_state = 42)
random_forest.fit(X_train, y_train)


print_score(random_forest ,X_train,y_train,X_test,y_test, train = True)
print_score(random_forest ,X_train,y_train,X_test,y_test, train = False)

print("time: %.2f" % (time()-start))

----------------------------------------------------------------------------------------------------
Training result:

Accuracy Score: 0.9931

Classification Report:
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      3005
          1       1.00      0.95      0.98       495

avg / total       0.99      0.99      0.99      3500


Confusion Matrix:
[[3004    1]
 [  23  472]]

Average Accuracy:	0.9457

Standard Deviation:	0.0100
----------------------------------------------------------------------------------------------------
Test result:

Accuracy Score: 0.9600

Classification Report:
             precision    recall  f1-score   support

          0       0.96      1.00      0.98      1288
          1       0.96      0.75      0.84       212

avg / total       0.96      0.96      0.96      1500


Confusion Matrix:
[[1282    6]
 [  54  158]]

time: 3.74
